<a href="https://colab.research.google.com/github/shilz1007/shilz1007/blob/main/Imp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries 
## connect to GPU
## Hyperparameter 
## transform normalize
## Import datasets (image and result)
## divide train and test 
## create loader
## create model
## create loss 
## Create optimizer 
## loop through epochs

In [1]:
import os
import copy
import time
import numpy as np
import pandas as pd
import torch.nn as nn
import torch
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from pandas import io 
from torch.utils.data import Dataset,DataLoader
from torchvision.io import read_image
import matplotlib.pyplot as plt
import skimage.io
from PIL import Image
from PIL import Image, ImageDraw
import random
%matplotlib inline


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
path1 = '/content/drive/MyDrive/SCD/Validation/GroundTruth.csv'
path2 = '/content/drive/MyDrive/SCD/Images'
#!ls {path2}

In [5]:
df = pd.read_csv(path1)
rows = len(df.index)
rows

10015

In [ ]:
for ind in range(rows):
  df['image'][ind] = df['image'][ind] +'.jpg'
  print(df['image'][ind])

In [7]:
df.head()

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
0,ISIC_0024306.jpg,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0024307.jpg,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0024308.jpg,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0024309.jpg,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0024310.jpg,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
class SkinData(Dataset):
  def __init__(self,csv_file,root_dir,transform=None):
    self.annotations = csv_file
    self.root_dir = root_dir
    self.transform = transform
  def __len__(self):
    return len(self.annotations)

  def __getitem__(self, index):
  
    img_path = os.path.join(self.root_dir,self.annotations.iloc[index,0])
     #image = skimage.io.imread(img_path)
    image_new = read_image(img_path)
    y_label = torch.tensor(int(self.annotations.iloc[index, 1]))

    #if self.transform:
    #   image_new = self.transform(image_new,y_label)

    return (image_new,y_label)  


     


 

In [9]:
## hyper parameters
import numpy as np

in_channel = 3
num_classes = 2
learning_rate = 0.001
b_size = 100
num_epochs = 50

mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

data_transforms = {
      transforms.Compose([
      transforms.RandomResizedCrop(224),
      transforms.RandomHorizontalFlip(),
      transforms.ToTensor(),
      transforms.Normalize(mean, std)                          
    ]),
}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# load data
## load data 
#csv_file = '/content/drive/MyDrive/GroundTruth.csv'
dataset =  SkinData(csv_file = df,
                    root_dir= '/content/drive/MyDrive/SCD/Images',
                    transform=data_transforms) 

data_size = (len(dataset)) 

train_set ,test_set = torch.utils.data.random_split(dataset,[7011,3004])



train_loader = DataLoader(train_set, batch_size = b_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size = b_size, shuffle=True)

#-------------------------------------------
#train_features, train_labels = next(iter(train_loader))
#print(f"Feature batch shape: {train_features.size()}")
#print(f"Labels batch shape: {train_labels.size()}")
#img = train_features[0].squeeze()
#label = train_labels[0]
#plt.imshow(img)
#plt.show()
#print(f"Label: {label}")


In [ ]:
#df['MEL'].count()
pd.value_counts(df['MEL'])
## out of 10015 images 8902 are benign and 1113 are malignant

0.0    8902
1.0    1113
Name: MEL, dtype: int64

In [14]:
def train_model(model,criterion,optimizer,schedular,num_epochs=50):
    since = time.time()

    best_model_weights = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train_set','test_set']:
            if phase == 'train':
               model.train()
            else:
               model.eval()
            running_loss = 0.0
            running_corrects = 0

            for inputs , lables in enumerate(train_loader):
               inputs = train_set.to(device)
               labels = labels.to(device)

               optimizer.zero_grad()

               with torch.set_grad_enabled(phase=='train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs,1)
                    loss = criterion(outputs,labels)

                    if phase == 'train':
                      optimizer.zero_grad()
                      loss.backward()
                      optimizer.step()

               running_loss += loss.item() * input.size(0)
               running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
              schedular.step()

            epoch_loss = running_loss / data_size[phase]
            epoch_acc = running_corrects.double() / data_size[phase]
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            if phase == 'val' and epoch_acc > best_acc:
               best_acc = epoch_acc
               best_model_wts = copy.deepcopy(model.state_dict())
        print()
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    model.load_state_dict(best_model_wts)
    return model

In [15]:
model_ft = torchvision.models.resnext50_32x4d(pretrained = True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs,2)
model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(),lr=learning_rate,momentum = 0.9)
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft , step_size=7,gamma=0.1)



In [16]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=5)


Epoch 0/4
----------


AttributeError: ignored

In [ ]:
class block(nn.Module):
    def __init__(self,in_channel,middle_channel,identity_change=None,stride=1):
      super.__init__(block,self)
      self.multiply = 4
      self.conv1 = mm.Conv2d(in_channels,middle_channel,kernal_size=1,stride=1,padding=0,bias=False)
      self.batchN1 = nn.BatchNorm2d(middle_channel)
      self.conv2 = nn.Conv2d(middle_channel,middle_channel,kernel_size=3,stride=stride,padding=1,bias=False))
      self.batchN2 = nn.BatchNorm2d(middle_channel)
      self.conv3 = nn.Conv2d(middle_channel,middle_channel * self.multiply,kernel_size = 1, stride=1,padding=0,bias=False)
      self.batchN3 = nn.BatchNorm2d(middle_channel*self.multiply)
      self.relu = nn.ReLU()
      self.identity_change = identity_change
      self.stride = stride 

    def forward(self,x):
        identity = x.clone()

        x = self.conv1(x)
        x = self.batchN1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.batchN2(x) 
        x = self.relu(x)
        x = self.conv3(x)
        x = self.batchN2(x)
        x = self.relu(x)

        if self.identity_change is not None:
           identity = self.identity_change(identity)

        x += identity
        x = relu(x)

        return x    
